In [55]:
import pandas as pd
import dask.dataframe as dd
import requests
import tempfile
from io import StringIO
import chardet
from unidecode import unidecode

# URLs de los archivos en el bucket S3 público
base_url = "https://prueba-fruitycert.s3.us-east-1.amazonaws.com/"
file_names = [
    "AtributosCliente.csv",
    "TablonInspecciones.csv",
    "ParametrosInspeccion.csv",
    "AtributosLotes.csv"
]

# Diccionario para almacenar los DataFrames
dataframes = {}

# Especificar los tipos de datos para las columnas problemáticas en TablonInspecciones.csv
dtypes = {
    'Analysis2': 'object',
    'Analysis4': 'object',
    'Cat': 'object',
    'Importer': 'object',
    'NumeroMuestra': 'float64',
    'OriginRegion': 'object',
    'Packing': 'object',
    'PurchaseOrder': 'object',
    'ShippingOrder': 'object',
    'ValorParametroInspeccion': 'object',
    'Warehouse': 'object'
}

for file_name in file_names:
    file_url = f"{base_url}{file_name}"
    try:
        # Descargar contenido del archivo ignorando la verificación SSL
        response = requests.get(file_url, verify=False)
        response.raise_for_status()  # Lanza un error si el status no es 200

        # Detectar codificación del archivo
        raw_data = response.content
        result = chardet.detect(raw_data)
        encoding_detected = result['encoding']
        print(f"Codificación detectada para '{file_name}': {encoding_detected}")

        if file_name == "TablonInspecciones.csv":
            # Guardar en un archivo temporal y luego leer con Dask
            with tempfile.NamedTemporaryFile(delete=False, suffix=".csv") as temp_file:
                temp_file.write(raw_data)
                temp_file_path = temp_file.name

            # Leer el archivo con los tipos de datos especificados
            ddf = dd.read_csv(temp_file_path, dtype=dtypes, encoding=encoding_detected)
            dataframes[file_name] = ddf
            print(f"Archivo '{file_name}' cargado exitosamente con Dask.")
        else:
            # Cargar archivos pequeños como DataFrame completo con Pandas
            df = pd.read_csv(StringIO(raw_data.decode(encoding_detected)), encoding=encoding_detected)

            # Normalizar los nombres de columnas y datos
            df.columns = [unidecode(col) for col in df.columns]
            df = df.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)

            dataframes[file_name] = df
            print(f"Archivo '{file_name}' cargado exitosamente con Pandas.")
    except Exception as e:
        print(f"Error al cargar '{file_name}': {e}")






/Users/claudiosanzocares/Desktop/Python/Postulación Microservice/lab_prueba_fruitycert/ambiente_lab_fruitycert/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prueba-fruitycert.s3.us-east-1.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Codificación detectada para 'AtributosCliente.csv': utf-8
Archivo 'AtributosCliente.csv' cargado exitosamente con Pandas.


/var/folders/y7/7bd167nn23v7p8cm41cgwfrw0000gn/T/ipykernel_11386/2653859983.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)
/Users/claudiosanzocares/Desktop/Python/Postulación Microservice/lab_prueba_fruitycert/ambiente_lab_fruitycert/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prueba-fruitycert.s3.us-east-1.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Codificación detectada para 'TablonInspecciones.csv': utf-8
Archivo 'TablonInspecciones.csv' cargado exitosamente con Dask.


/Users/claudiosanzocares/Desktop/Python/Postulación Microservice/lab_prueba_fruitycert/ambiente_lab_fruitycert/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prueba-fruitycert.s3.us-east-1.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Codificación detectada para 'ParametrosInspeccion.csv': utf-8
Archivo 'ParametrosInspeccion.csv' cargado exitosamente con Pandas.


/var/folders/y7/7bd167nn23v7p8cm41cgwfrw0000gn/T/ipykernel_11386/2653859983.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)


Codificación detectada para 'AtributosLotes.csv': ascii
Archivo 'AtributosLotes.csv' cargado exitosamente con Pandas.


/Users/claudiosanzocares/Desktop/Python/Postulación Microservice/lab_prueba_fruitycert/ambiente_lab_fruitycert/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'prueba-fruitycert.s3.us-east-1.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/var/folders/y7/7bd167nn23v7p8cm41cgwfrw0000gn/T/ipykernel_11386/2653859983.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)


In [56]:
dask_df = dataframes["TablonInspecciones.csv"]
result = dask_df.describe().compute()
print(result)


          IdPlanilla         Fecha  FechaPublicacion      InspDate  \
count  610230.000000  6.102300e+05      6.102300e+05  6.102300e+05   
mean    12301.250753  2.022054e+07      2.022055e+07  2.022054e+07   
std      6211.295763  6.395906e+02      3.567175e+02  6.395906e+02   
min      3466.000000  2.021100e+07      2.022010e+07  2.021100e+07   
25%      8675.000000  2.022030e+07      2.022030e+07  2.022030e+07   
50%      9744.000000  2.022043e+07      2.022043e+07  2.022043e+07   
75%     20747.000000  2.022093e+07      2.022093e+07  2.022093e+07   
max     26634.000000  2.023010e+07      2.022123e+07  2.023010e+07   

            ArrDate      PackDate  HarvestDate  ReceiptDate  Store  \
count  3.044000e+03  4.344910e+05          0.0          0.0    0.0   
mean   1.964319e+07  2.019157e+07          NaN          NaN    NaN   
std    3.240463e+06  6.812701e+05          NaN          NaN    NaN   
min    1.092022e+06  2.022023e+06          NaN          NaN    NaN   
25%    2.022080e+07

In [57]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 10) 
dask_df.dtypes

IdCliente                     object
NombreCliente                 object
IdTipoInspeccion              object
NombreTipoInspeccion          object
IdEspecie                     object
NombreEspecie                 object
IdPlanilla                     int64
LugarInspeccion               object
Fecha                          int64
FechaPublicacion               int64
IdUnidad                      object
InspDate                       int64
ArrDate                      float64
PackDate                     float64
HarvestDate                  float64
ReceiptDate                  float64
Exporter                      object
Importer                      object
Grower                        object
Market                        object
Port                          object
Packing                       object
Warehouse                     object
Store                        float64
Farm                         float64
FarmPlot                     float64
Variety                       object
S

In [75]:
# dask_df[dask_df['IdPlanilla'] == 15348].tail(100)
dask_df[dask_df['IdUnidad'] == '800100384263'].tail(100)

,IdCliente,NombreCliente,IdTipoInspeccion,NombreTipoInspeccion,IdEspecie,NombreEspecie,IdPlanilla,LugarInspeccion,Fecha,FechaPublicacion,IdUnidad,InspDate,ArrDate,PackDate,HarvestDate,ReceiptDate,Exporter,Importer,Grower,Market,Port,Packing,Warehouse,Store,Farm,FarmPlot,Variety,Size,Cat,Label,Package,Product,OriginRegion,Vessel,Container,PurchaseOrder,ShippingOrder,Analysis1,Analysis2,Analysis3,Analysis4,Analysis5,CantidadSubunidades,NumeroMuestra,CodigoParametroInspeccion,ValorParametroInspeccion
301597,tropifresh,TropiFresh,LAVE,Inspeccion de Arribo,LEMONS,LEMONS,15348,WILMINGTON,20220609,20220524,800100384263,20220609,NaN,NaN,NaN,NaN,SAN MIGUEL,NaN,SAN MIGUEL,philadelphia,WILMINGTON,NaN,NaN,NaN,NaN,NaN,EUREKA,165,NaN,SAN MIGUEL,17.2KG,NaN,NaN,MONTE VERDE,SUDU5159400,NaN,NaN,22ALM002_SANMIGUEL_SUDU5159400,NaN,NaN,NaN,NaN,72,11.0,4,800100384263
301598,tropifresh,TropiFresh,LAVE,Inspeccion de Arribo,LEMONS,LEMONS,15348,WILMINGTON,20220609,20220524,800100384263,20220609,NaN,NaN,NaN,NaN,SAN MIGUEL,NaN,SAN MIGUEL,philadelphia,WILMINGTON,NaN,NaN,NaN,NaN,NaN,EUREKA,165,NaN,SAN MIGUEL,17.2KG,NaN,NaN,MONTE VERDE,SUDU5159400,NaN,NaN,22ALM002_SANMIGUEL_SUDU5159400,NaN,NaN,NaN,NaN,72,11.0,5,72
301599,tropifresh,TropiFresh,LAVE,Inspeccion de Arribo,LEMONS,LEMONS,15348,WILMINGTON,20220609,20220524,800100384263,20220609,NaN,NaN,NaN,NaN,SAN MIGUEL,NaN,SAN MIGUEL,philadelphia,WILMINGTON,NaN,NaN,NaN,NaN,NaN,EUREKA,165,NaN,SAN MIGUEL,17.2KG,NaN,NaN,MONTE VERDE,SUDU5159400,NaN,NaN,22ALM002_SANMIGUEL_SUDU5159400,NaN,NaN,NaN,NaN,72,11.0,6,17.2
301644,tropifresh,TropiFresh,LAVE,Inspeccion de Arribo,LEMONS,LEMONS,15348,WILMINGTON,20220609,20220524,800100384263,20220609,NaN,NaN,NaN,NaN,SAN MIGUEL,NaN,SAN MIGUEL,philadelphia,WILMINGTON,NaN,NaN,NaN,NaN,NaN,EUREKA,165,NaN,SAN MIGUEL,17.2KG,NaN,NaN,MONTE VERDE,SUDU5159400,NaN,NaN,22ALM002_SANMIGUEL_SUDU5159400,NaN,NaN,NaN,NaN,72,11.0,COND,2
301645,tropifresh,TropiFresh,LAVE,Inspeccion de Arribo,LEMONS,LEMONS,15348,WILMINGTON,20220609,20220524,800100384263,20220609,NaN,NaN,NaN,NaN,SAN MIGUEL,NaN,SAN MIGUEL,philadelphia,WILMINGTON,NaN,NaN,NaN,NaN,NaN,EUREKA,165,NaN,SAN MIGUEL,17.2KG,NaN,NaN,MONTE VERDE,SUDU5159400,NaN,NaN,22ALM002_SANMIGUEL_SUDU5159400,NaN,NaN,NaN,NaN,72,11.0,DALE00,4
301646,tropifresh,TropiFresh,LAVE,Inspeccion de Arribo,LEMONS,LEMONS,15348,WILMINGTON,20220609,20220524,800100384263,20220609,NaN,NaN,NaN,NaN,SAN MIGUEL,NaN,SAN MIGUEL,philadelphia,WILMINGTON,NaN,NaN,NaN,NaN,NaN,EUREKA,165,NaN,SAN MIGUEL,17.2KG,NaN,NaN,MONTE VERDE,SUDU5159400,NaN,NaN,22ALM002_SANMIGUEL_SUDU5159400,NaN,NaN,NaN,NaN,72,11.0,DALE01,165
301647,tropifresh,TropiFresh,LAVE,Inspeccion de Arribo,LEMONS,LEMONS,15348,WILMINGTON,20220609,20220524,800100384263,20220609,NaN,NaN,NaN,NaN,SAN MIGUEL,NaN,SAN MIGUEL,philadelphia,WILMINGTON,NaN,NaN,NaN,NaN,NaN,EUREKA,165,NaN,SAN MIGUEL,17.2KG,NaN,NaN,MONTE VERDE,SUDU5159400,NaN,NaN,22ALM002_SANMIGUEL_SUDU5159400,NaN,NaN,NaN,NaN,72,11.0,DALE02,NaN
301648,tropifresh,TropiFresh,LAVE,Inspeccion de Arribo,LEMONS,LEMONS,15348,WILMINGTON,20220609,20220524,800100384263,20220609,NaN,NaN,NaN,NaN,SAN MIGUEL,NaN,SAN MIGUEL,philadelphia,WILMINGTON,NaN,NaN,NaN,NaN,NaN,EUREKA,165,NaN,SAN MIGUEL,17.2KG,NaN,NaN,MONTE VERDE,SUDU5159400,NaN,NaN,22ALM002_SANMIGUEL_SUDU5159400,NaN,NaN,NaN,NaN,72,11.0,DALE06,165
301649,tropifresh,TropiFresh,LAVE,Inspeccion de Arribo,LEMONS,LEMONS,15348,WILMINGTON,20220609,20220524,800100384263,20220609,NaN,NaN,NaN,NaN,SAN MIGUEL,NaN,SAN MIGUEL,philadelphia,WILMINGTON,NaN,NaN,NaN,NaN,NaN,EUREKA,165,NaN,SAN MIGUEL,17.2KG,NaN,NaN,MONTE VERDE,SUDU5159400,NaN,NaN,22ALM002_SANMIGUEL_SUDU5159400,NaN,NaN,NaN,NaN,72,11.0,DALE11,10
301650,tropifresh,TropiFresh,LAVE,Inspeccion de Arribo,LEMONS,LEMONS,15348,WILMINGTON,20220609,20220524,800100384263,20220609,NaN,NaN,NaN,NaN,SAN MIGUEL,NaN,SAN MIGUEL,philadelphia,WILMINGTON,NaN,NaN,NaN,NaN,NaN,EUREKA,165,NaN,SAN MIGUEL,17.2KG,NaN,NaN,MONTE VERDE,SUDU5159400,NaN,NaN,22ALM002_SANMIGUEL_SUDU5159400,NaN,NaN,NaN,NaN,72,11.0,DALE13,4


In [59]:
pd.set_option('display.max_rows', None) 
print(dataframes['AtributosCliente.csv'].shape)
print(dataframes['AtributosCliente.csv'].head(1000))

(2086, 3)
       IdCliente      IdAtributo          NombreAtributo
0       fruitexp       Analysis1          Numero de Lote
1       fruitexp       Analysis2              Contenedor
2       fruitexp       Analysis3               Nro Hatch
3       fruitexp       Analysis4              PO Cliente
4       fruitexp       Analysis5            Subproductor
5       fruitexp         ArrDate            Fecha Arribo
6       fruitexp           Store            Supermercado
7       fruitexp         Variety                Variedad
8       fruitexp          Vessel               PO / Nave
9       fruitexp       Warehouse  Frigorifico o Locacion
10      fruitexp            Port                  Puerto
11      fruitexp   PurchaseOrder         Orden de Compra
12      fruitexp     ReceiptDate         Fecha Recepcion
13      fruitexp   ShippingOrder          Orden de Venta
14      fruitexp            Size                 Calibre
15      fruitexp          Specie                 Especie
16      fruitexp     

In [ ]:
dataframes['AtributosLotes.csv'].head()

,IdCliente,IdTipoInspeccion,IdEspecie,InspDate,ArrDate,PackDate,HarvestDate,ReceiptDate,Exporter,Importer,Grower,Market,Port,Packing,Warehouse,Store,Farm,FarmPlot,Variety,Size,Cat,Label,Package,Product,OriginRegion,Vessel,Container,PurchaseOrder,ShippingOrder,Analysis1,Analysis2,Analysis3,Analysis4,Analysis5,ParamConcat
0,exotifruit,AVO2,AVOCADO,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,Label;Variety;Grower;Package;Cat;Size
1,exotifruit,AVOC,AVOCADO,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,Label;Variety;Grower;Package;Cat;Size
2,exotifruit,CITR,CITRUS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,Variety;Grower;Package;Label;Size;Cat
3,exotifruit,LAVE,AVOCADO,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,Label;Variety;Grower;Package;Cat;Size
4,exotifruit,LAVE,MANGOES,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,Package;Label;Variety;Grower;Size;Cat


In [24]:
dataframes['AtributosLotes.csv'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   IdCliente         78 non-null     object
 1   IdTipoInspeccion  78 non-null     object
 2   IdEspecie         78 non-null     object
 3   InspDate          78 non-null     int64 
 4   ArrDate           78 non-null     int64 
 5   PackDate          78 non-null     int64 
 6   HarvestDate       78 non-null     int64 
 7   ReceiptDate       78 non-null     int64 
 8   Exporter          78 non-null     int64 
 9   Importer          78 non-null     int64 
 10  Grower            78 non-null     int64 
 11  Market            78 non-null     int64 
 12  Port              78 non-null     int64 
 13  Packing           78 non-null     int64 
 14  Warehouse         78 non-null     int64 
 15  Store             78 non-null     int64 
 16  Farm              78 non-null     int64 
 17  FarmPlot          

In [80]:
print(dataframes['ParametrosInspeccion.csv'].shape)
dataframes['ParametrosInspeccion.csv'].head(1000)
dataframes['ParametrosInspeccion.csv'][dataframes['ParametrosInspeccion.csv'].duplicated(subset = ['IdTipoInspeccion','IdEspecie','CodigoParametroInspeccion'],keep=False)].sort_values(by=['IdTipoInspeccion','IdEspecie','CodigoParametroInspeccion'])

(2721, 8)


,IdCliente,IdTipoInspeccion,IdEspecie,CodigoParametroInspeccion,PrefijoParametroInspeccion,NombreParametroInspeccion,EsDefecto,NombrePrefijo
696,fruitexp,LAVE,CHERRIES,4,AAAA,Pallet Nro,False,Identificacion
2596,tropifresh,LAVE,CHERRIES,4,AAAA,Pallet Nro,False,Identificacion
697,fruitexp,LAVE,CHERRIES,5,AAAA,Cajas/Pallet,False,Identificacion
2597,tropifresh,LAVE,CHERRIES,5,AAAA,Cases/Pallet,False,Identificacion
698,fruitexp,LAVE,CHERRIES,6,AAAA,Peso Neto Caja,False,Identificacion
2598,tropifresh,LAVE,CHERRIES,6,AAAA,Case Net Weight,False,Identificacion
699,fruitexp,LAVE,CHERRIES,7,AAAA,Fecha Embalaje,False,Identificacion
2599,tropifresh,LAVE,CHERRIES,7,AAAA,Packing Date,False,Identificacion
700,fruitexp,LAVE,CHERRIES,COND,SCOR,Condicion,False,Calificacion
2600,tropifresh,LAVE,CHERRIES,COND,SCOR,Condition,False,Calificacion


In [63]:
dataframes['TablonInspecciones.csv'].columns

Index(['IdCliente', 'NombreCliente', 'IdTipoInspeccion',
       'NombreTipoInspeccion', 'IdEspecie', 'NombreEspecie', 'IdPlanilla',
       'LugarInspeccion', 'Fecha', 'FechaPublicacion', 'IdUnidad', 'InspDate',
       'ArrDate', 'PackDate', 'HarvestDate', 'ReceiptDate', 'Exporter',
       'Importer', 'Grower', 'Market', 'Port', 'Packing', 'Warehouse', 'Store',
       'Farm', 'FarmPlot', 'Variety', 'Size', 'Cat', 'Label', 'Package',
       'Product', 'OriginRegion', 'Vessel', 'Container', 'PurchaseOrder',
       'ShippingOrder', 'Analysis1', 'Analysis2', 'Analysis3', 'Analysis4',
       'Analysis5', 'CantidadSubunidades', 'NumeroMuestra',
       'CodigoParametroInspeccion', 'ValorParametroInspeccion'],
      dtype='object')